Source:
http://seba1511.net/tutorials/beginner/nlp/word_embeddings_tutorial.html#annotations:L7uIBvyoEem7yT_qOQZJ0A

This is what we mean by a notion of similarity: we mean semantic similarity, not simply having similar orthographic representations. It is a technique to combat the sparsity of linguistic data, by connecting the dots between what we have seen and what we haven’t. This example of course relies on a fundamental linguistic assumption: that words appearing in similar contexts are related to each other semantically. This is called the distributional hypothesis.

### Getting Dense Word Embeddings

https://hyp.is/rGnQWABrEequN0vinIOvdw/seba1511.net/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [6]:
# Dictionary wordToIx is a mapping from words to indices

## index = the unique index for a word in a word embedding

## Embeddings are stored as: |V| x D matrix, where 
### D = dimensionality of the embeddings such that the word
# assigned index i has its embedding stored in the ith
# row of the matrix
### |V| = dimension of the vocabulary V

wordToIx = {"hello":0, "world": 1}

# 2 words in vocab, 5 dimensional embeddings
embeds = nn.Embedding(2, 5)

# Getting the word embedding for the word "hello" using
# the lookup dictionary's index 0
lookupTensor = torch.LongTensor([wordToIx["hello"]])
helloEmbed = embeds(autograd.Variable(lookupTensor))

print(helloEmbed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]], grad_fn=<EmbeddingBackward>)


## Example: N-Gram Language Modelling

from IPython.display import Latex

Latex(r"""

\begin{equation}
P(w_i | w_{i-1}, w_{i-2}, ..., w_{i - n+1})
\end{equation}

""")
